In [8]:
import pandas as pd
import joblib
from cryptography.fernet import Fernet
import hashlib
import torch
from transformers import DistilBertTokenizer

In [11]:
# # 2. Charger le modèle et le tokenizer avec joblib
# # Use torch.load with map_location to force loading on CPU
# model = joblib.load("distilbert_model.joblib")
# model = model.to(torch.device('cpu'))  # Ensure the model is loaded to the CPU
# tokenizer = joblib.load("tokenizer.joblib")

In [10]:
# Charger la clé et le modèle
with open("fernet.key", "rb") as f:
    key = f.read()
fernet = Fernet(key)

In [2]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Charger le modèle et le tokenizer
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Load the state dict into the model, ensuring it's on the CPU
model.load_state_dict(torch.load("distilbert_model.pth", map_location=torch.device('cpu')))
model.to(torch.device('cpu'))  # Ensure the model is on CPU

tokenizer = joblib.load("tokenizer_new.joblib")  # Load the tokenizer

In [6]:


# 3. Fonction de prédiction
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred_class = torch.argmax(probs).item()
    label = "positive" if pred_class == 1 else "negative"
    return label, probs[0][pred_class].item()

# 4. Exemple d'utilisation
msg = "I absolutely loved this movie. It was amazing!"
label, confidence = predict_sentiment(msg)

print(f"🎬 Message : {msg}")
print(f"✅ Prédiction : {label} ({confidence:.2%} confiance)")


🎬 Message : I absolutely loved this movie. It was amazing!
✅ Prédiction : positive (99.81% confiance)


In [14]:
# Charger les critiques chiffrées
df = pd.read_csv("encrypted_imdb.csv")

# Générer un ID utilisateur incrémental
df['user_id'] = ['employee_' + str(i) for i in range(len(df))]



In [15]:
# Ajouter une colonne "role" pour simuler les rôles des utilisateurs
# Rôle 0 pour Data Scientist et rôle 1 pour Data Analyst
df['role'] = [0 if i % 2 == 0 else 1 for i in range(len(df))]

# Fonction pour appliquer un chiffrement ou hachage selon le rôle
def process_user_id(row):
    if row['role'] == 1:  # Data Analyst : Chiffrement et Hachage
        # Hachage de l'ID utilisateur
        user_hash = hashlib.sha256(row['user_id'].encode()).hexdigest()
        # Chiffrement de l'ID utilisateur
        user_encrypted = fernet.encrypt(row['user_id'].encode()).decode()
        return pd.Series([user_hash, user_encrypted], index=['user_hash', 'user_encrypted'])
    else:  # Data Scientist : Pas de chiffrement
        return pd.Series([row['user_id'], None], index=['user_hash', 'user_encrypted'])

In [16]:
# Appliquer la fonction pour gérer les ID utilisateur
df[['user_hash', 'user_encrypted']] = df.apply(process_user_id, axis=1)

# Fonction pour déchiffrer et prédire la polarité du message
def predict_sentiment(row):
    # Déchiffrer le message
    encrypted_msg = row['encrypted_text']
    msg = fernet.decrypt(encrypted_msg.encode()).decode()

    # Prétraitement du message pour le tokenizer DistilBERT
    inputs = tokenizer(msg, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Prédire la polarité (positif/négatif) en utilisant le modèle DistilBERT
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()  # 0 for negative, 1 for positive

    return 'positive' if prediction == 1 else 'negative'

In [18]:
# Appliquer la prédiction
df['prediction'] = df.apply(predict_sentiment, axis=1)

# Sauvegarder le fichier 'access_log' pour l'application Streamlit
df.to_csv('access_log.csv', index=False)

print("✅ Fichier 'access_log' exporté")

# Vérification du contenu du fichier généré
print(df[['user_id', 'user_hash', 'user_encrypted', 'imdb_text', 'prediction']].head())
